# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [174]:
# import numpy and pandas

import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import t, sem


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [73]:
# Your code here:

salaries = pd.read_csv("C:/Users/TG/Desktop/IronHack_Data/lab-hypothesis-testing-1/your-code/Current_Employee_Names__Salaries__and_Position_Titles.csv")

Examine the `salaries` dataset using the `head` function below.

In [83]:
# Your code here:
salaries.columns = data.columns.str.replace(" ", "_")
salaries.head()

,Name,Job_Titles,Department,Full_or_Part-Time,Salary_or_Hourly,Typical_Hours,Annual_Salary,Hourly_Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [129]:
# Your code here:

salaries.isnull().sum()

Name                     0
Job_Titles               0
Department               0
Full_or_Part-Time        0
Salary_or_Hourly         0
Typical_Hours        25161
Annual_Salary         8022
Hourly_Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [130]:
# Your code here:

salaries["Salary_or_Hourly"].value_counts()

Salary    25161
Hourly     8022
Name: Salary_or_Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [86]:
# Your code here:

departments = salaries.pivot_table(index="Department", 
                     values="Name",
                     aggfunc="count").sort_values(by="Name", 
                                                 ascending=False)

departments.head(5)

#or salaries["Department"].value_counts().head(1)

,Name
Department,
POLICE,13414
FIRE,4641
STREETS & SAN,2198
OEMC,2102
WATER MGMNT,1879


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [186]:
# Your code here:

hourly = salaries.query("Salary_or_Hourly == 'Hourly'")
print(f'Average wage is:',hourly["Hourly_Rate"].mean())

#H0: p = 30
#H1: p != 30

st.ttest_1samp(hourly["Hourly_Rate"],30)

#pvalue < 2.5%

Average wage is: 32.78855771628024


Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [168]:
# Your code here:

police = salaries.query("Department == 'POLICE'")
print(f'Average wage is:',police["Annual_Salary"].mean())


#H0: p <= 86000
#H1: p > 86000

stat, pvalue = st.ttest_1samp(police["Annual_Salary"],86000, nan_policy='omit')

print("stat:", stat, "p-value:", pvalue/2)


Average wage is: 86486.41450313339
stat: 3.081997005712994 p-value: 0.00103017017755027


Using the `crosstab` function, find the department that has the most hourly workers. 

In [167]:
# Your code here:

most_hourly = pd.crosstab(salaries.Department, salaries.Typical_Hours)
most_hourly.loc[:,'Row_Total'] = most_hourly.sum(numeric_only=True, axis=1)
most_hourly.sort_values('Row_Total', ascending=False)

Typical_Hours,10.0,20.0,35.0,40.0,Row_Total
Department,,,,,
STREETS & SAN,176,0,10,1676,1862
WATER MGMNT,0,0,0,1513,1513
OEMC,0,1229,0,44,1273
AVIATION,0,18,15,1049,1082
GENERAL SERVICES,0,0,0,765,765
TRANSPORTN,0,0,0,725,725
PUBLIC LIBRARY,0,288,0,11,299
FAMILY & SUPPORT,0,286,0,1,287
CITY COUNCIL,7,36,18,3,64


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [169]:
# Your code here:

most_hourly_wk = salaries.query("Department == 'STREETS & SAN'")
print(f'Average wage is:',most_hourly_wk["Hourly_Rate"].mean())

#H0: p >= 35
#H1: p < 35 

stat, pvalue = st.ttest_1samp(most_hourly_wk["Hourly_Rate"],35, nan_policy='omit')

print("stat:", stat, "p-value:", pvalue/2)

Average wage is: 33.72837808807734
stat: -9.567447887848152 p-value: 1.6689265282355369e-21


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [182]:
# Your code here:

t.interval(0.95, 
           (salaries["Hourly_Rate"].count()-1), 
           hourly["Hourly_Rate"].mean(), 
           sem(salaries["Hourly_Rate"], 
            nan_policy='omit') )

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [183]:
# Your code here:

t.interval(0.95, 
           (police["Annual_Salary"].count()-1), 
           police["Annual_Salary"].mean(), 
           sem(police["Annual_Salary"], 
            nan_policy='omit') )

(86177.05631531784, 86795.77269094894)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

